In [1]:
using CSV, DataFrames, JuMP, Gurobi, Formatting, NPZ

# Loading Data

In [2]:
time_s2d_df = CSV.read("csv/time_srcs_to_dests.csv", DataFrame, header=false);
length_s2d_df = CSV.read("csv/length_srcs_to_dests.csv", DataFrame, header=false);

In [3]:
first(time_s2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,7.129,48.7983,31.9467,22.8094,26.2942,22.6127,28.3366,21.3805,20.7362
2,35.44,17.4854,5.28,27.8,26.9882,22.2878,11.64,24.9489,24.9139


In [4]:
first(length_s2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.742265,5.07616,3.27882,2.46325,2.77267,2.36016,2.88682,2.36068,2.20435
2,3.7094,1.8535,0.551457,2.95893,2.83249,2.28854,1.24267,2.65683,2.6224


In [5]:
n_src = nrow(time_s2d_df);
n_dst = ncol(time_s2d_df);

all_src = 1:n_src;
all_dst = 1:n_dst;

println("all_src: $all_src, all_dst: $all_dst")

all_src: 1:3, all_dst: 1:9


In [6]:
T = Matrix(time_s2d_df);
L = Matrix(length_s2d_df);

println("T: $(size(T)), L: $(size(L))")

T: (3, 9), L: (3, 9)


# Problem
- We now additionally consider that each robot has a fixed starting source.

In [7]:
S = [5, 1, 3];
@assert size(S) == (n_src,)
@assert sum(S) == n_dst

# Minimum Time

In [8]:
model1 = Model(Gurobi.Optimizer)

@variable(model1, x[all_src, all_dst] >= 0);

@objective(model1, Min, sum(sum(T[ii, jj] * x[ii, jj] for ii in all_src) for jj in all_dst));

# Each destination must be served by at least one source.
@constraint(model1, demand[jj in all_dst], sum(x[ii, jj] for ii in all_src) >= 1);

# There is one robot for each destination.
@constraint(model1, nrobots, sum(sum(x[ii, jj] for ii in all_src) for jj in all_dst) == n_dst);

# Each robot must start from its assigned source.
@constraint(model1, start[ii in all_src], sum(x[ii, jj] for jj in all_dst) == S[ii]);

# Solve.
optimize!(model1);
solution_summary(model1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 3950X 16-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 13 rows, 27 columns and 81 nonzeros
Model fingerprint: 0xff95c60c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 27 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0880007e+01   6.500000e+00   0.000000e+00      0s
       9    1.4863289e+02   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.486328908e+02

User-callback calls 67, time in user-callback 0.00

* Solver : Gurobi

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.48633e+02
  Objective bound    : 1.48633e+02
  Dual objective value : 1.48633e+02

* Work counters
  Solve time (sec)   : 3.39031e-04
  Barrier iterations : 0
  Node count         : 0


In [9]:
# Save solution.
x1 = Array(value.(x));
npzwrite("sols/problem2_time.npz", Dict("x" => x1, "objective" => objective_value(model1)));

x1

3×9 Matrix{Float64}:
 1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0

# Minimum Distance

In [10]:
model2 = Model(Gurobi.Optimizer)

@variable(model2, x[all_src, all_dst] >= 0);

@objective(model2, Min, sum(sum(L[ii, jj] * x[ii, jj] for ii in all_src) for jj in all_dst));

# Each destination must be served by at least one source.
@constraint(model2, demand[jj in all_dst], sum(x[ii, jj] for ii in all_src) >= 1);

# There is one robot for each destination.
@constraint(model2, nrobots, sum(sum(x[ii, jj] for ii in all_src) for jj in all_dst) == n_dst);

# Each robot must start from its assigned source.
@constraint(model2, start[ii in all_src], sum(x[ii, jj] for jj in all_dst) == S[ii]);

# Solve.
optimize!(model2);
solution_summary(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 3950X 16-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 13 rows, 27 columns and 81 nonzeros
Model fingerprint: 0xfd0b7d87
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 27 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2500401e+00   6.500000e+00   0.000000e+00      0s
      10    1.5815293e+01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.581529286e+01

User-callback calls 67, time in user-callback 0.0

* Solver : Gurobi

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.58153e+01
  Objective bound    : 1.58153e+01
  Dual objective value : 1.58153e+01

* Work counters
  Solve time (sec)   : 2.28882e-04
  Barrier iterations : 0
  Node count         : 0


In [11]:
# Save solution.
x2 = Array(value.(x));
npzwrite("sols/problem2_dist.npz", Dict("x" => x2, "objective" => objective_value(model2)));

x2

3×9 Matrix{Float64}:
 1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0